In [1]:
import cv2
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
from tensorflow.keras.models import Model

In [2]:
# Google Drive 마운트
drive.mount('/content/drive')

# 데이터 경로 설정
data_path = '/content/drive/MyDrive/kfood'

Mounted at /content/drive


In [3]:
# 사전 훈련된 ResNet50 모델 로드
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델 구성
model = Sequential()
model.add(base_model)  # ResNet50을 기본 모델로 추가
model.add(layers.GlobalAveragePooling2D())  # 평균 풀링 층
model.add(layers.Dense(64, activation='relu'))  # 커스텀 분류 층
model.add(layers.Dropout(0.5))  # 드롭아웃 층
model.add(layers.Dense(10, activation='softmax'))  # 최종 출력 층

# 기본 모델의 층을 학습 가능하게 설정
base_model.trainable = True

94765736/94765736 [==============================] - 0s 0us/step


In [4]:
# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 23719498 (90.48 MB)
Trainable params: 23666378 (90.28 MB)
Non-trainable params: 53120 (207.50 KB)
__________

In [6]:
# 데이터 제너레이터 생성
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# 훈련 데이터셋 제너레이터 설정
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_path, 'train'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)  # 훈련 데이터셋 셔플

# 검증 데이터셋 제너레이터 설정
validation_generator = val_datagen.flow_from_directory(
    os.path.join(data_path, 'val'),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)  # 검증 데이터셋 셔플하지 않음

# 모델 학습
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator
)

Found 7882 images belonging to 10 classes.
Found 1991 images belonging to 10 classes.
Epoch 1/30
247/247 [==============================] - 2855s 11s/step - loss: 1.4978 - accuracy: 0.5199 - val_loss: 212.4886 - val_accuracy: 0.1005
Epoch 2/30
247/247 [==============================] - 97s 392ms/step - loss: 1.0275 - accuracy: 0.6873 - val_loss: 7.1483 - val_accuracy: 0.1005
Epoch 3/30
247/247 [==============================] - 107s 433ms/step - loss: 0.8139 - accuracy: 0.7672 - val_loss: 2.8051 - val_accuracy: 0.2290
Epoch 4/30
247/247 [==============================] - 98s 396ms/step - loss: 0.5998 - accuracy: 0.8169 - val_loss: 1.1681 - val_accuracy: 0.6665
Epoch 5/30
247/247 [==============================] - 98s 395ms/step - loss: 0.4470 - accuracy: 0.8627 - val_loss: 1.5218 - val_accuracy: 0.6429
Epoch 6/30
247/247 [==============================] - 98s 397ms/step - loss: 0.3969 - accuracy: 0.8844 - val_loss: 2.5588 - val_accuracy: 0.6143
Epoch 7/30
247/247 [=====================

In [ ]:
# Epoch별 train/val accuracy와 Loss변화

import matplotlib.pyplot as plt

def plot_history(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_history(history)

NameError: ignored

## 평가 및 시각화


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 모든 검증 데이터에 대한 예측 수행
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# 실제 레이블 가져오기
y_true = validation_generator.classes

# 클래스 이름 가져오기
class_names = list(validation_generator.class_indices.keys())

# 혼동 행렬 그리기
plot_confusion_matrix(y_true, y_pred_classes, classes=class_names)

In [ ]:
from sklearn.metrics import classification_report

# Accuracy, Precision, Recall, F1-Score 계산
print(classification_report(y_true, y_pred_classes, target_names=class_names))

In [ ]:
# 모델 저장
model.save("resnet_1st.h5")

# 파일 크기 확인
import os
model_size = os.path.getsize("resnet_1st.h5")
print(f"Model Size: {model_size / 1024 / 1024:.2f} MB")

In [ ]:
import time

# 샘플 데이터 준비 (예: 검증 데이터셋의 첫 번째 배치)
X_sample, _ = next(iter(validation_generator))

# 추론 시간 측정
start_time = time.time()
predictions = model.predict(X_sample)
end_time = time.time()

print(f"Prediction Time: {end_time - start_time:.4f} seconds")

In [ ]:
import random

# 무작위로 100개의 인덱스 선택
indices = random.sample(range(len(validation_generator.filenames)), 100)

# 시각화
for i in indices:
    img_path = os.path.join(validation_generator.directory, validation_generator.filenames[i])
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    true_label = class_names[y_true[i]]
    pred_label = class_names[y_pred_classes[i]]

    plt.figure(figsize=(2, 2))
    plt.imshow(img)
    plt.title(f'Predicted: {pred_label}\nActual: {true_label}',
              color='green' if true_label == pred_label else 'red')
    plt.axis('off')
    plt.show()